In [17]:
from pypokerengine.players import BasePokerPlayer
import pypokerengine.utils.visualize_utils as U
class FishPlayer(BasePokerPlayer):  # Do not forget to make parent class as "BasePokerPlayer"

    #  we define the logic to make an action through this method. (so this method would be the core of your AI)
    def declare_action(self, valid_actions, hole_card, round_state, msg):
        # valid_actions format => [raise_action_info, call_action_info, fold_action_info]
        #print(hole_card)
        #print(round_state['community_card'])
        #print(round_state)
        #print(valid_actions)
        #print(msg['player'].paid_sum())
        
        call_action_info = valid_actions[1]
        action, amount = call_action_info["action"], call_action_info["amount"]
        return action, amount   # action returned here is sent to the poker engine

    def receive_game_start_message(self, game_info):
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        #print(seats)
        pass

    def receive_street_start_message(self, street, round_state):
        #print(street)
        pass

    def receive_game_update_message(self, action, round_state):
        #print(action)
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        #print('\n\n\n')
        #print(hand_info)
        #print(U.visualize_round_result(winners, hand_info, round_state, self.uuid))
        #print(hand_info)
        pass
    

In [18]:
from pypokerengine.api.game import setup_config, start_poker

config = setup_config(max_round=10, initial_stack=100, small_blind_amount=5)
config.register_player(name="p1", algorithm=FishPlayer())
config.register_player(name="p2", algorithm=FishPlayer())
config.register_player(name="p3", algorithm=FishPlayer())
game_result = start_poker(config, verbose=1)

Started the round 1
Street "preflop" started. (community card = [])
"p1" declared "call:10"
"p2" declared "call:10"
"p3" declared "call:10"
Street "flop" started. (community card = ['D4', 'HQ', 'D6'])
"p2" declared "call:0"
"p3" declared "call:0"
"p1" declared "call:0"
Street "turn" started. (community card = ['D4', 'HQ', 'D6', 'HJ'])
"p2" declared "call:0"
"p3" declared "call:0"
"p1" declared "call:0"
Street "river" started. (community card = ['D4', 'HQ', 'D6', 'HJ', 'S5'])
"p2" declared "call:0"
"p3" declared "call:0"
"p1" declared "call:0"
"['p2']" won the round 1 (stack = {'p2': 120, 'p3': 90, 'p1': 90})
Started the round 2
Street "preflop" started. (community card = [])
"p2" declared "call:10"
"p3" declared "call:10"
"p1" declared "call:10"
Street "flop" started. (community card = ['H6', 'D5', 'CQ'])
"p3" declared "call:0"
"p1" declared "call:0"
"p2" declared "call:0"
Street "turn" started. (community card = ['H6', 'D5', 'CQ', 'SJ'])
"p3" declared "call:0"
"p1" declared "call:0"
"

In [9]:
nb_simulation = 1000
nb_player = 3
hole_card = ['H4', 'D7']
community_card = ['D3', 'C5', 'C6']

def estimate_hand_strength(nb_simulation, nb_player, hole_card, community_card):
    simulation_results = []
    for i in range(nb_simulation):
        opponents_cards = []
        for j in range(nb_player-1):  # nb_opponents = nb_player - 1
            opponents_cards.append(draw_cards_from_deck(num=2))
        nb_need_community = 5 - len(community_card)
        community_card.append(draw_cards_from_deck(num=nb_need_community))
        result = observe_game_result(hole_card, community_card, opponents_cards)  # return 1 if win else 0
        simulation_results.append(result)
    average_win_rate = 1.0 * sum(simulation_results) / len(simulation_results)
    return average_win_rate

In [4]:
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate
hole_card = gen_cards(['H4', 'D7'])
community_card = gen_cards(['D3', 'C5', 'C6'])
estimate_hole_card_win_rate(nb_simulation=1000, nb_player=3, hole_card=hole_card, community_card=community_card)

estimate_hole_card_win_rate(nb_simulation=1000, nb_player=3, hole_card=hole_card, community_card=community_card)



0.845

a = BasePokerPlayer()
a.nb_player

In [19]:
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate

NB_SIMULATION = 1000

class HonestPlayer(BasePokerPlayer):

    def declare_action(self, valid_actions, hole_card, round_state, msg):
        community_card = round_state['community_card']
        bblind = self.game_info['rule']['small_blind_amount']*2
        win_rate = estimate_hole_card_win_rate(
                nb_simulation=NB_SIMULATION,
                nb_player=self.nb_player,
                hole_card=gen_cards(hole_card),
                community_card=gen_cards(community_card)
                )
        paid = msg['player'].paid_sum()
        #print(paid)
        #print(hole_card, win_rate, (valid_actions[1]['amount'] - paid),(bblind*self.nb_player) , (valid_actions[1]['amount'] - paid)/(bblind*self.nb_player))
        #print('\n')
        
        
        if win_rate >= (valid_actions[1]['amount'] - paid)/(bblind*self.nb_player):
            action = valid_actions[1]  # fetch CALL action info
        else:
            action = valid_actions[0]  # fetch FOLD action info
        return action['action'], action['amount']

    def receive_game_start_message(self, game_info):
        self.nb_player = game_info['player_num']
        self.game_info = game_info

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

In [ ]:
from pypokerengine.api.game import setup_config, start_poker
config = setup_config(max_round=1000, initial_stack=1000, small_blind_amount=5)
config.register_player(name="fish_player_1", algorithm=FishPlayer())
config.register_player(name="fish_player_2", algorithm=FishPlayer())
config.register_player(name="fish_player_3", algorithm=FishPlayer())
config.register_player(name="fish_player_4", algorithm=FishPlayer())
config.register_player(name="honest_player_1", algorithm=HonestPlayer())
config.register_player(name="honest_player_2", algorithm=HonestPlayer())
config.register_player(name="honest_player_3", algorithm=HonestPlayer())
config.register_player(name="honest_player_4", algorithm=HonestPlayer())
game_result = start_poker(config, verbose=1)
for player_info in game_result["players"]:
     print (player_info)

Started the round 1
Street "preflop" started. (community card = [])
"fish_player_4" declared "call:10"
"honest_player_1" declared "call:10"
"honest_player_2" declared "call:10"
"honest_player_3" declared "fold:0"
"honest_player_4" declared "call:10"
"fish_player_1" declared "call:10"
"fish_player_2" declared "call:10"
"fish_player_3" declared "call:10"
Street "flop" started. (community card = ['S6', 'C8', 'C2'])
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
"honest_player_1" declared "call:0"
"honest_player_2" declared "call:0"
"honest_player_4" declared "call:0"
"fish_player_1" declared "call:0"
Street "turn" started. (community card = ['S6', 'C8', 'C2', 'H4'])
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
"honest_player_1" declared "call:0"
"honest_player_2" declared "call:0"
"honest_player_4" declared "call:0"
"fish_player_1" declared "call:0"
Street "river" started. (comm

"honest_player_3" declared "call:0"
"honest_player_4" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
"honest_player_2" declared "call:0"
"['fish_player_1']" won the round 6 (stack = {'honest_player_4': 1035, 'fish_player_2': 940, 'honest_player_2': 1015, 'fish_player_1': 1010, 'fish_player_4': 1010, 'honest_player_1': 970, 'fish_player_3': 1060, 'honest_player_3': 960})
Started the round 7
Street "preflop" started. (community card = [])
"fish_player_2" declared "call:10"
"fish_player_3" declared "call:10"
"fish_player_4" declared "call:10"
"honest_player_1" declared "call:10"
"honest_player_2" declared "fold:0"
"honest_player_3" declared "fold:0"
"honest_player_4" declared "call:10"
"fish_player_1" declared "call:10"
Street "flop" started. (community card = ['D4', 'HT', 'H2'])
"honest_player_4" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"

"honest_player_1" declared "call:0"
"honest_player_2" declared "call:0"
"honest_player_3" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
Street "river" started. (community card = ['D6', 'H9', 'S9', 'HJ', 'DQ'])
"honest_player_1" declared "call:0"
"honest_player_2" declared "call:0"
"honest_player_3" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
"['honest_player_1']" won the round 12 (stack = {'honest_player_4': 1135, 'fish_player_2': 940, 'honest_player_2': 1065, 'fish_player_1': 990, 'fish_player_4': 950, 'honest_player_1': 990, 'fish_player_3': 1000, 'honest_player_3': 930})
Started the round 13
Street "preflop" started. (community card = [])
"honest_player_4" declared "call:10"
"fish_player_1" declared "call:10"
"fish_player_2" declared "call:10"
"fish_player_3" declared 

"honest_player_4" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
Street "turn" started. (community card = ['C8', 'S2', 'D8', 'DJ'])
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
"honest_player_4" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
Street "river" started. (community card = ['C8', 'S2', 'D8', 'DJ', 'CQ'])
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
"honest_player_4" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"['fish_player_3']" won the round 18 (stack = {'honest_player_4': 1155, 'fish_player_2': 880, 'honest_player_2': 1045, 'fish_player_1': 980, 'fish_player_4': 950, 'honest_player_1': 1050, 'fish_player_3': 1030, 'honest_player_3': 910})
Started the round 19
Street "preflop" started. (community card = [])
"honest_player_2" declared "call:10"
"honest_player_3" declared "fold:0"
"honest_player_4" declared "fold:

"honest_player_1" declared "call:10"
"honest_player_2" declared "fold:0"
"honest_player_3" declared "call:10"
"honest_player_4" declared "fold:0"
"fish_player_1" declared "call:10"
"fish_player_2" declared "call:10"
Street "flop" started. (community card = ['HT', 'S5', 'HK'])
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
"honest_player_1" declared "call:0"
"honest_player_3" declared "call:0"
Street "turn" started. (community card = ['HT', 'S5', 'HK', 'C7'])
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
"honest_player_1" declared "call:0"
"honest_player_3" declared "call:0"
Street "river" started. (community card = ['HT', 'S5', 'HK', 'C7', 'D5'])
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
"honest_player_1" declared "ca

...

"honest_player_1" declared "call:0"
"honest_player_2" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
Street "turn" started. (community card = ['D4', 'D3', 'SA', 'ST'])
"honest_player_1" declared "call:0"
"honest_player_2" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
Street "river" started. (community card = ['D4', 'D3', 'SA', 'ST', 'C7'])
"honest_player_1" declared "call:0"
"honest_player_2" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
"['fish_player_3']" won the round 36 (stack = {'honest_player_4': 1130, 'fish_player_2': 890, 'honest_player_2': 1005, 'fish_player_1': 1000, 'fish_player_4': 885, 'honest_player_1': 1150, 'fish_player_3': 1120, 'honest_player_3': 820}

"honest_player_1" declared "fold:0"
"honest_player_2" declared "call:10"
"honest_player_3" declared "fold:0"
"honest_player_4" declared "fold:0"
"fish_player_1" declared "call:10"
"fish_player_2" declared "call:10"
"fish_player_3" declared "call:10"
"fish_player_4" declared "call:10"
Street "flop" started. (community card = ['H4', 'D8', 'SQ'])
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
"honest_player_2" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
Street "turn" started. (community card = ['H4', 'D8', 'SQ', 'DA'])
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
"honest_player_2" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
Street "river" started. (community card = ['H4', 'D8', 'SQ', 'DA', 'H2'])
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
"honest_player_2" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:

"honest_player_4" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
"['fish_player_3']" won the round 47 (stack = {'honest_player_4': 1135, 'fish_player_2': 780, 'honest_player_2': 1125, 'fish_player_1': 1020, 'fish_player_4': 905, 'honest_player_1': 1160, 'fish_player_3': 1060, 'honest_player_3': 815})
Started the round 48
Street "preflop" started. (community card = [])
"fish_player_3" declared "call:10"
"fish_player_4" declared "call:10"
"honest_player_1" declared "call:10"
"honest_player_2" declared "fold:0"
"honest_player_3" declared "call:10"
"honest_player_4" declared "call:10"
"fish_player_1" declared "call:10"
"fish_player_2" declared "call:10"
Street "flop" started. (community card = ['HJ', 'HA', 'D2'])
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
"honest_player_1" declared "call:0"
"h

"honest_player_1" declared "fold:0"
"honest_player_2" declared "call:10"
"honest_player_3" declared "call:10"
Street "flop" started. (community card = ['SK', 'S9', 'DK'])
"honest_player_2" declared "call:0"
"honest_player_3" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
Street "turn" started. (community card = ['SK', 'S9', 'DK', 'S8'])
"honest_player_2" declared "call:0"
"honest_player_3" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
Street "river" started. (community card = ['SK', 'S9', 'DK', 'S8', 'D3'])
"honest_player_2" declared "call:0"
"honest_player_3" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
"fish_player_4" declared "call:0"
"['honest_player_2']" won the round 53 (stack = {'honest_player_

"honest_player_4" declared "call:10"
"fish_player_1" declared "call:10"
"fish_player_2" declared "call:10"
"fish_player_3" declared "call:10"
"fish_player_4" declared "call:10"
"honest_player_1" declared "call:10"
Street "flop" started. (community card = ['CK', 'S3', 'HQ'])
"fish_player_4" declared "call:0"
"honest_player_1" declared "call:0"
"honest_player_4" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
Street "turn" started. (community card = ['CK', 'S3', 'HQ', 'S4'])
"fish_player_4" declared "call:0"
"honest_player_1" declared "call:0"
"honest_player_4" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "call:0"
"fish_player_3" declared "call:0"
Street "river" started. (community card = ['CK', 'S3', 'HQ', 'S4', 'HJ'])
"fish_player_4" declared "call:0"
"honest_player_1" declared "call:0"
"honest_player_4" declared "call:0"
"fish_player_1" declared "call:0"
"fish_player_2" declared "ca